In [ ]:
pip install OpenAI

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install whisper-mic

     ---------------------------------------- 60.8/60.8 kB 3.4 MB/s eta 0:00:00
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
     ---------------------------------------- 57.0/57.0 kB ? eta 0:00:00
     -------------------------------------- 798.6/798.6 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
     ---------------------------------------- 89.2/89.2 kB 4.9 MB/s eta 0:00:00
  Using cached rich-13.7.0-py3-none-any.whl (240 kB)
     --------------------------------------- 32.8/32.8 MB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     --------


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


pip install transformers

In [21]:
pip install faster-whisper pyaudio wave transformers torchaudio pyttsx3


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



     ------------------------------------- 194.1/194.1 kB 12.3 MB/s eta 0:00:00


In [1]:
from openai import OpenAI
from faster_whisper import WhisperModel
import pyaudio
import wave
import os
import pyttsx3

e:\coding\pythonLearning\MachineLearning\ImageClassification\imageClassification\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import logging
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import pyaudio
import numpy as np

logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s %(message)s')
logging.info("wav2vec_microphone.main()")

model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
device = "cuda" if torch.cuda.is_available() else 'cpu'

model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)
processor = Wav2Vec2Processor.from_pretrained(model_name)

sampling_rate = 16000
record_time_in_seconds = 5
number_of_samples = round(record_time_in_seconds * sampling_rate)

def stt():
    mic_stream = pyaudio.PyAudio().open(format=pyaudio.paInt16,
                            channels=1,
                            rate=sampling_rate,
                            input=True,
                            frames_per_buffer=number_of_samples)

    # Get audio
    logging.info("Speak!")
    speech_arr = np.frombuffer(mic_stream.read(number_of_samples), dtype=np.int16)
    print(speech_arr)

    speech_tsr = torch.from_numpy(speech_arr)
    # Tokenize our tensor
    input_values = processor(speech_tsr, return_tensord="pt", sampling_rate=sampling_rate)["input_values"]
    input_tsr = torch.from_numpy(input_values[0]).to(device).unsqueeze(0)

    # Perform inference
    logits = model(input_tsr)["logits"]
    predicted_ids = torch.argmax(logits, dim=-1)
    logging.info(f"type(predicted_ids) = {type(predicted_ids)}; predicted_ids.shape = {predicted_ids.shape}")

    # Decode the IDs to text
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription


2024-02-10 21:55:38,657 INFO wav2vec_microphone.main()
Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.po

In [3]:
engine = pyttsx3.init()
    
rate = engine.getProperty('rate')   # getting details of current speaking rate
engine.setProperty('rate', 200)

2024-02-10 21:55:41,198 INFO Imported existing <module 'comtypes.gen' from 'e:\\coding\\pythonLearning\\MachineLearning\\ImageClassification\\imageClassification\\Lib\\site-packages\\comtypes\\gen\\__init__.py'>
2024-02-10 21:55:41,199 INFO Using writeable comtypes cache directory: 'e:\coding\pythonLearning\MachineLearning\ImageClassification\imageClassification\Lib\site-packages\comtypes\gen'


In [4]:
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

def main():
    history = [
        {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."},
        {"role": "user", "content": "Hello, introduce yourself to someone opening this program for the first time. Be concise."},
    ]

    while True:
        completion = client.chat.completions.create(
            model="local-model", # this field is currently unused
            messages=history,  
            temperature=0.7,
            stream=True,
        )

        new_message = {"role": "assistant", "content": ""}
        end_msg = 0
        
        for chunk in completion:
            if chunk.choices[0].delta.content:
                print(chunk.choices[0].delta.content, end="")       
                new_message["content"] += chunk.choices[0].delta.content
                if (new_message["content"].find('.') != -1):
                        engine.say(new_message["content"][end_msg:len(new_message["content"])])
                        engine.runAndWait()
                        end_msg += len(new_message["content"])
        history.append(new_message)

        print()
        history.append({"role": "user", "content": stt()})

main()

2024-02-10 21:55:56,286 INFO HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"


Hi there! I'm an AI assistant designed to help you with a variety of tasks. How can I assist you today?

2024-02-10 21:56:03,191 INFO Speak!



[-32 -62 -62 ...  32  33  33]


C:\Users\admin\AppData\Local\Temp\ipykernel_30328\266110212.py:33: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:212.)
  speech_tsr = torch.from_numpy(speech_arr)
2024-02-10 21:56:12,076 INFO type(predicted_ids) = <class 'torch.Tensor'>; predicted_ids.shape = torch.Size([1, 249])
2024-02-10 21:56:12,095 INFO HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"


...

2024-02-10 21:56:35,426 INFO type(predicted_ids) = <class 'torch.Tensor'>; predicted_ids.shape = torch.Size([1, 249])
2024-02-10 21:56:35,445 INFO HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"


Angela Woltman is an American writer who is best known for her science fiction novel "The Tomorrow War." The book tells the story of a scientist named Dr. Cathy Fish who works on a top-secret government project and becomes embroiled in a war with alien invaders. The novel has received positive reviews for its engaging plot, well-developed characters, and thought-provoking themes. If you enjoyed "The Tomorrow War," you may also want to check out other science fiction novels by Angela Woltman or similar authors

2024-02-10 21:56:51,918 INFO Speak!


.
[ 11  20  17 ... 100  99  97]


: 